source: https://www.magicbricks.com/property-for-rent-in-new-delhi-pppfr/

In [ ]:
#pages 1-363.....

In [1]:
import re
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [12]:
print('pages 1-362:', round(os.stat('NEW_DELHI_1-362.txt').st_size*1e-6, 2), 'mb')

pages 1-362: 268.84 mb


In [13]:
with open('NEW_DELHI_1-362.txt', 'r') as data:
    data = [page+'/html>' for page in data.read().split('/html>')[:-1]]

In [14]:
def page_to_df(data):
    failed=0
    rows=[]
    soup = BeautifulSoup(data)
    for listing in soup.find_all("div", id=re.compile("^resultBlockWrapper")):
        try: # check if listing has price
            new_row = {'price': ' '.join(listing.find('div', {'class':'m-srp-card__price'}).text.split()),
                       'title': ' '.join(listing.find('span',{'class':'m-srp-card__title'}).text.split()),}
            try:new_row.update({coord.attrs['itemprop']: coord.attrs['content']
                               for coord in listing.find('div',{'itemprop':'geo'}).find_all('meta')})
            except:pass

            try:new_row.update({summary_card.find_all('div')[0].text:summary_card.find_all('div')[1].text.strip()
                               for summary_card in listing.find_all('div', {'class':'m-srp-card__summary__item'})})
            except:pass
            rows.append(new_row)
        except: failed+=1
            # skip if no price.
    df = pd.DataFrame(rows)
    return len(rows), failed, df

In [21]:
success = 0
failed = 0
df = pd.DataFrame()
print(len(data),'pages to add...')
for i, page in enumerate(data):
    page_to_df(data[i])
    page_success, page_failed, page_df = page_to_df(data[i])
    success+=page_success
    failed+=page_failed
    df = pd.concat([df, page_df], sort=False)
    if i%30==0: print(f'{i} pages added...')
print('COMPLETED.')
print(f'{success} listings added.\n{failed} listings not added')
df

362 pages to add...
0 pages added...
30 pages added...
60 pages added...
90 pages added...
120 pages added...
150 pages added...
180 pages added...
210 pages added...
240 pages added...
270 pages added...
300 pages added...
330 pages added...
360 pages added...
COMPLETED.
8416 listings added.
464 listings not added


,price,title,longitude,latitude,floor,furnishing,tenants preferred,bathroom,balcony,availability,facing,overlooking,car parking,owner resides
0,"₹ 10,000",1 BHK Builder Floor for rent in Saket,28.523548126220703,77.20889282226562,2 out of 4 Floors,Furnished,Bachelors/Family,2,1,Immediately,NaN,NaN,NaN,NaN
1,₹ 1.6 Lac,4 BHK Builder Floor for rent in Greater Kailas...,NaN,NaN,2 out of 3 Floors,Semi-Furnished,Bachelors/Family,4,2,Immediately,South,"Garden/Park, Main Road","1 Covered, 1 Open",NaN
2,"₹ 7,000",1 BHK Builder Floor for rent in Chhattarpur,NaN,NaN,2 out of 3 Floors,Semi-Furnished,Bachelors/Family,1,NaN,Immediately,East,Main Road,1 Open,NaN
3,"₹ 20,500","1 BHK House for rent in Dayanand Colony, Lajpa...",28.562599182128906,77.24960327148438,Ground out of 3 Floors,Furnished,Bachelors/Family,1,1,Immediately,NaN,NaN,NaN,NaN
4,"₹ 13,000",1 BHK Apartment for rent in Rohini Sector 18,28.74116325378418,77.13461303710938,2 out of 4 Floors,Semi-Furnished,Bachelors/Family,1,1,Immediately,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,"₹ 20,000",2 BHK Apartment for rent in Dwarka Sector 14 7...,NaN,NaN,3 out of 4 Floors,Semi-Furnished,Bachelors/Family,2,NaN,Immediately,NaN,NaN,NaN,NaN
24,"₹ 55,000",3 BHK Apartment for rent in Tower Height Apart...,28.7001547,77.1509864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,"₹ 9,000",1 BHK Apartment for rent in Rohini Sector 18,NaN,NaN,3 out of 4 Floors,Semi-Furnished,Bachelors,1,1,Immediately,North - East,Garden/Park,2 Open,NaN
26,"₹ 16,000",2 BHK Apartment for rent in Millennium Apartme...,28.7365307170796,77.134687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
print(len(df), '--->', len(df)-sum(df.duplicated()))
df.drop_duplicates(inplace=True)

8416 ---> 2346


In [23]:
df.to_csv('NEW_DELHI_1-362_data.csv', index=False)